In [1]:
# ==============================================================================
# 🚀 MASTER SCRIPT: ESG AGENT (PORTFOLIO VERSION)
# ==============================================================================

# --- 1. IMPORTS ---
import os
import asyncio
from dotenv import load_dotenv
from azure.identity import AzureCliCredential
from agent_framework.azure import AzureOpenAIChatClient
from agent_framework import ChatAgent

print("🔄 1. Initializing environment setup...")

# --- 2. ENVIRONMENT CONFIGURATION ---
load_dotenv()

endpoint_model = os.getenv("AZURE_MODEL_ENDPOINT")
deployment = os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME")
model_key = os.getenv("AZURE_AI_MODEL_API_KEY")

print("✅ Environment variables loaded securely.")

# --- 3. TOOL DEFINITION (With Validation) ---
def calculate_employment_cost(number_of_people: int, contract_months: int, minimum_wage: float) -> str:
    """
    Calculates the total cost of a green employment program.
    The agent MUST have this information to use the tool.
    """
    
    # --- INPUT VALIDATION (Guard Clauses) ---
    # Protecting the code against invalid data
    if number_of_people <= 0:
        return "Error: The number of people must be greater than zero."
    if contract_months <= 0:
        return "Error: The contract must be at least 1 month long."
    if minimum_wage < 0:
        return "Error: The wage cannot be negative."

    # --- BUSINESS LOGIC ---
    total_cost = number_of_people * contract_months * minimum_wage
    
    # Format the response (BRL Currency)
    result = f"The estimated cost to hire {number_of_people} people for {contract_months} months, at a wage of BRL {minimum_wage:,.2f}, is BRL {total_cost:,.2f}."
    
    print(f"   🛠️ [Tool Debug]: Calculation function triggered. Result generated.")
    return result

# --- 4. AGENT DEFINITION (With Guardrails) ---
async def create_esg_agent():
    # Azure Client Setup
    chat_client = AzureOpenAIChatClient(
        endpoint=endpoint_model,
        api_key=model_key,
        deployment_name=deployment,
        api_version="2024-02-15-preview", 
    )
    
    # Agent Setup
    esg_agent = ChatAgent(
        chat_client=chat_client,
        name="esg_cost_calculator_agent",
        
        # --- PROMPT ENGINEERING (Instructions) ---
        instructions=(
            "You are an expert assistant for ESG and Public Policy, focused on data from Brazil. "
            "Your job is to help public managers estimate the costs of green employment programs. "
            "\n\n"
            "--- WHEN TO USE THE TOOL ---\n"
            "When the user asks about costs, budgets, or estimates, "
            "you MUST use the tool 'calculate_employment_cost'. "
            "You need to collect 3 values: number_of_people, contract_months, and minimum_wage. "
            "\n\n"
            "--- SAFETY GUARDRAILS ---\n"
            "1. SCOPE: You ONLY answer about ESG and Public Policy. If asked about sports, recipes, or jokes, politely refuse.\n"
            "2. REALITY CHECK: If the values seem unrealistic (e.g., wage of 1 cent), warn the user."
        ),
        tools=[calculate_employment_cost] 
    )
    return esg_agent

# --- 5. EXECUTION (Robust) ---
async def run_esg_agent():
    print("\n🤖 2. Starting ESG Agent...")
    
    # Sample Question (English)
    user_question = "Hello! How much would it cost to hire 1,500 people for 6 months, at a minimum wage of 1518.00 BRL?"
    
    try:
        # Create Agent
        esg_agent = await create_esg_agent()
        
        print(f"📩 3. Sending question: '{user_question}'")
        print("⏳ 4. Processing (AI is thinking)...")
        
        # Run Agent
        result = await esg_agent.run(user_question)
        
        print("\n============================================")
        print("✅ FINAL ESG AGENT RESPONSE")
        print("============================================")
        print(result)
        print("============================================\n")
        print("🎉 Process Complete! Take your screenshot now!")
        
    except Exception as e:
        print(f"\n❌ Technical Error: {e}")

# --- TRIGGER ---
await run_esg_agent()

🔄 1. Initializing environment setup...
✅ Environment variables loaded securely.

🤖 2. Starting ESG Agent...
📩 3. Sending question: 'Hello! How much would it cost to hire 1,500 people for 6 months, at a minimum wage of 1518.00 BRL?'
⏳ 4. Processing (AI is thinking)...
   🛠️ [Tool Debug]: Calculation function triggered. Result generated.

✅ FINAL ESG AGENT RESPONSE
The estimated cost to hire 1,500 people for 6 months at a minimum wage of BRL 1,518.00 per month is BRL 13,662,000.00.

If you need further details or want cost breakdowns (e.g., including benefits or social charges), let me know!

🎉 Process Complete! Take your screenshot now!
